# Running Injury Clinic data

Reginaldo K Fukuchi, Feb 2023, reginaldo.fukuchi@ufabc.edu.br

This NB imports mat file containing joint angle data and save as txt files

In [1]:
# Prepare environment
import os, glob
import scipy.io as spio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
# Import data
dir_path = r'C:\Users\Reginaldo\Documents\data\CNPq\MCR_run'
fnames = glob.glob(os.path.join(dir_path,'*.mat'))
print('N= '+str(len(fnames))+' subjects')

N= 22 subjects


## Import mat file data
### Helper functions to prepare data
https://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries

In [4]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

## Change index of subject

In [248]:
fname = fnames[22]
print('Running...')
fname

IndexError: list index out of range

### Import mat file data

In [ ]:
# Import data
data = loadmat(fname)

In [ ]:
data['out'].keys()

In [ ]:
joints = list(data['out']['r_norm_ang'].keys())
joints = ['hip','knee','ankle']
axes = ['X','Y','Z']

In [ ]:
events = data['out']['r_events']
max_stride_time_R = np.max(np.diff(events, axis=0)[:,2])

## PENDING
* TIME NORMALIZE DATA
* AVERAGE ACROSS TRIALS
* SAVE AS TXT
* PICK SIMILAR SUBJECTS IN RBDS
* COMPARE THEM

In [ ]:
from tnorma import tnorma

In [ ]:
angs_R = np.empty(shape=(101, len(events)-1, len(joints), len(axes))) * np.NaN

for j, joint in enumerate(joints):
    angs = np.array(data['out']['r_angles']['R_'+joint])
    ang = np.empty(shape=(max_stride_time_R, len(events)-1))
    for i in range(len(events)-1):
        yn, tn, indie = tnorma(angs[events[i][2]:events[i+1][2], :], k=1,
                               smooth=0, mask=None, show=False)
        angs_R[:, i, j, :] = yn

### Plotting

In [ ]:
angs_Rm= np.empty(shape=(101, len(joints)*len(axes))) * np.NaN

fig, axs = plt.subplots(nrows=len(joints), ncols=len(axes), figsize=(10, 6))
plt.subplots_adjust(hspace=0.5)
fig.suptitle("Right Joint Angles", fontsize=18, y=0.95)
for j, joint in enumerate(joints):
    for xyz, eixos in enumerate(axes):
        axs[j, xyz].plot(tn, angs_R[:, :, j, xyz])
        axs[j, xyz].set_ylabel(joint)
        
    angs_Rm[:,3*j:3*(j+1)] = np.mean(angs_R[:, :, j, :], axis=1)# mean
    
plt.show()

## Create pandas df

In [ ]:
columns = [joint+'_'+eixo for joint in joints for eixo in axes]
dv_R = pd.DataFrame(data=angs_Rm, columns=columns)

In [ ]:
dv_R['Frame']=tn
dv_R.set_index('Frame', inplace=True)
dv_R.head()

In [ ]:
# Export to txt
dv_R.to_csv(os.path.join(dir_path, fname[-19:-4]+'.txt'), sep='\t')